In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import matplotlib.ticker as ticker
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from datetime import datetime, timedelta

In [ ]:
arr = []
for i in range(2):
    today = datetime.today() - timedelta(i * 30)
    req = requests.get('https://crix-api-endpoint.upbit.com/v1/crix/candles/minutes/240?code=CRIX.UPBIT.KRW-DOGE&count=180&to='+ today.strftime("%Y-%m-%d")+'%2024:00:00')
    arr += req.json()        

#json 데이터화
arr.reverse()
df = pd.DataFrame(arr)
print(df)


In [ ]:
#데이터 자르기
df['candleDateTimeKst'] = df['candleDateTimeKst'].str.slice(start=0,stop=19).to_numpy()
datachart = df['tradePrice']
time = df['candleDateTimeKst'].to_numpy()
length = len(datachart)-1


plt.figure(figsize=(24, 16))
plt.plot(time, datachart)
ax=plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(60))
plt.xlabel('time')
plt.ylabel('price')

In [61]:
window_size = 6

def seq2dataset(datachart,window_size):
    X = []; Y = []
    for i in range(len(datachart) - window_size):
        X.append(np.array(datachart.iloc[i:i+window_size]))
        Y.append(np.array(datachart.iloc[i+window_size]))
    return np.array(X), np.array(Y)



X, Y = seq2dataset(datachart,window_size);
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
split = int(len(X)*0.7)
x_train=X[0:split]; y_train=Y[0:split]
x_test=X[split:]; y_test=Y[split:]

print(x_train.shape)
print(x_test.shape)

model=Sequential()
model.add(LSTM(128,activation='relu',input_shape=(x_train.shape[1], x_train.shape[2], )))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam', metrics=['mae']) #mae를 사용해야 특이값에 덜 민감하다.
model.fit(x_train, y_train, epochs=100,batch_size=32,validation_data=(x_test,y_test),verbose=2)
model.summary()

#예측
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)


#실제 데이터
plt.figure(figsize=(24, 16))
plt.plot(time, datachart)

#훈련 결과
split_tp = split + window_size
plt.plot(np.arange(window_size, split_tp, 1), train_predict, color='g')

#테스트 결과
plt.plot(np.arange(split_tp, split_tp + len(test_predict), 1), test_predict, color='r')

plt.ylabel('Price')
ax=plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(60))
plt.show()

x_range = range(length - 15,length + 1)
plt.figure(figsize=(24, 16))
plt.plot(time[x_range], datachart[x_range],color='red')
plt.plot(time[x_range], test_predict[range(len(test_predict) - 16 ,len(test_predict))], color='blue')
ax=plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
plt.legend(['True prices', 'Predicted prices'], loc='best')
plt.show()

In [65]:
now = time[time.size - 1]
print("최근 시간 데이터 : ", now)


newdata = np.array(datachart.iloc[length + 1 - window_size:length+1])
newdata = newdata.reshape((1, newdata.shape[0], 1))

newPredict = model.predict(newdata)
print("최근 가격 데이터 : ", datachart[length])
print("4시간 뒤 가격 예상 : ", newPredict)
sub = round(newPredict[0][0] - datachart[length], 2)
rate = round((newPredict[0][0]/datachart[length] - 1) * 100, 2)
if(sub > 0):
  print("예상 시간별 손익 : ", "+"+ str(sub))
else:
  print("예상 시간별 손익 : ", str(sub))
print("예상 수익률 : ", str(rate))

최근 시간 데이터 :  2021-06-02T13:00:00
최근 가격 데이터 :  441.0
4시간 뒤 가격 예상 :  [[435.33856]]
예상 시간별 손익 :  -5.66
예상 수익률 :  -1.28
